In [1]:
import pandas as pd
import numpy as np

from glob import glob
from dfply import *
from tqdm.auto import tqdm

In [2]:
import ee
ee.Initialize()

In [3]:
# define boundary and zones
china = ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/China_zone")

name_en2cn = dict(zip(['dongbei','huabei','huadong','xibei','xinan','zhongnan'],
                     ['东北','华北','华东','西北','西南','中南']))

# load terrain
DEM = ee.ImageCollection("COPERNICUS/DEM/GLO30").select('DEM').mosaic().toInt16()
SLOPE = ee.Terrain.slope(DEM).toInt8()
terrain = DEM.addBands(SLOPE)

In [4]:
# the drive can hold data of 4-year range
begin_yr = np.arange(1990,2023,3)
all_yrs = dict(enumerate([f'{yr}_{yr+2}' for yr in begin_yr]))
all_yrs

{0: '1990_1992',
 1: '1993_1995',
 2: '1996_1998',
 3: '1999_2001',
 4: '2002_2004',
 5: '2005_2007',
 6: '2008_2010',
 7: '2011_2013',
 8: '2014_2016',
 9: '2017_2019',
 10: '2020_2022'}

In [5]:
# read and concat all url_csv to one df
files = glob('./gee_asset_china/*.csv')
satellite_df = pd.concat([pd.read_csv(f) for f in files]).sort_values(['region','type','year']).reset_index(drop=True)

# create terrain df
years,regions = satellite_df['year'].unique(),satellite_df['region'].unique()
terrain_dict = [{'year':'1990_2020', 'type':'terrain', 'path':terrain, 'region':region} for region in regions]
terrain_df = pd.DataFrame(terrain_dict)

df = pd.concat([satellite_df,terrain_df],axis=0).sort_values(['year','region','type']).reset_index(drop=True)
df

,year,type,path,region
0,1990_1992,Fourier,projects/earthengine-legacy/assets/users/bthom...,dongbei
1,1990_1992,Landsat_cloud_free,projects/earthengine-legacy/assets/users/bthom...,dongbei
2,1990_1992,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/bthom...,dongbei
3,1990_1992,Fourier,projects/earthengine-legacy/assets/users/ldiaz...,huabei
4,1990_1992,Landsat_cloud_free,projects/earthengine-legacy/assets/users/ldiaz...,huabei
...,...,...,...,...
199,2020_2022,Landsat_cloud_free,projects/earthengine-legacy/assets/users/jinzh...,xinan
200,2020_2022,NDVI_NDBI_EVI,projects/earthengine-legacy/assets/users/jinzh...,xinan
201,2020_2022,Fourier,projects/earthengine-legacy/assets/users/cnbui...,zhongnan
202,2020_2022,Landsat_cloud_free,projects/earthengine-legacy/assets/users/cnbui...,zhongnan


In [6]:
# download_df
# download_df = df >> mask(X.year.isin([v for k,v in all_yrs.items() if ((k >= 4)&(k <= 7)) ]))

download_df = df >> mask(X.type == 'terrain')
download_df

,year,type,path,region
18,1990_2020,terrain,"ee.Image({\n ""functionInvocationValue"": {\n ...",dongbei
19,1990_2020,terrain,"ee.Image({\n ""functionInvocationValue"": {\n ...",huabei
20,1990_2020,terrain,"ee.Image({\n ""functionInvocationValue"": {\n ...",huadong
21,1990_2020,terrain,"ee.Image({\n ""functionInvocationValue"": {\n ...",xibei
22,1990_2020,terrain,"ee.Image({\n ""functionInvocationValue"": {\n ...",xinan
23,1990_2020,terrain,"ee.Image({\n ""functionInvocationValue"": {\n ...",zhongnan


In [37]:
for idx,row in tqdm(download_df.iterrows(),total=len(download_df)):
    
    img = ee.Image(row['path'])
    boundary = china.filter(ee.Filter.eq('NAME1',name_en2cn[row['region']])).geometry().bounds()
    desc = f"{row['region']}_{row['type']}_{row['year']}"
    
    task = ee.batch.Export.image.toDrive(
      image=img, 
      description=desc, 
      folder='GEE_tif', 
      fileNamePrefix=desc,
      region=boundary, 
      scale=30, 
      maxPixels=1e13, 
      skipEmptyTiles=True)
    
    task.start()

  0%|          | 0/66 [00:00<?, ?it/s]